In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from easyenergy.callbacks.keras import TrainCallback
from easyenergy.callbacks.keras import PredictCallback
from easyenergy.callbacks.keras import TestCallback

from easyenergy.callbacks.keras import PerEpochCallback

from easyenergy.callbacks.keras import TrainBatchCallback
from easyenergy.callbacks.keras import PredictBatchCallback

/Users/abhijithneilabraham/mambaforge/envs/py/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
#load btc data

In [ ]:
url = 'https://drive.google.com/'
url = url + 'uc?export=download&id=1_sFCLx-c6A_xsDAtl6FrHhdUDRDyJ0Or'
data = pd.read_csv(url)
btc_data = data[data['ticker'] == 'BTC']
btc_data_0 = btc_data[data['price_btc'] == 0]
drop_columns_list = btc_data.nunique()[btc_data.nunique() <= 2].index
btc_data.drop(drop_columns_list, axis=1, inplace=True)

In [ ]:
# Define data preprocessing method

In [ ]:
def data_preproc_and_split(data, n):
    col = []

    for i in range(n):
        col.append('price' + str(i))
        col.append('volume' + str(i))

    train = pd.DataFrame(columns=col)
    target = pd.DataFrame(columns=['date', 'price'])
    pred_convert = pd.DataFrame(columns=['date', 'price'])

    # Preprocessing of data
    for i in range(1, len(data)-n-1):
        def_nom = data.loc[i-1, 'price_usd']

        for j in range(n):
            train.loc[i, 'price' + str(j)] = data.loc[i+j,
                                                      'price_usd']/def_nom-1
            vstr = 'volume' + str(j)
            train.loc[i, vstr] = data.loc[i+j, 'volume']/data.loc[i+j,
                                                                  'market_cap']

        target.loc[i, 'price'] = data.loc[i+n+1, 'price_usd']/def_nom-1
        target.loc[i, 'date'] = data.loc[i+n+1, 'trade_date']
        # Save start prices for convertation prediction resalt to valid prices
        pred_convert.loc[i, 'price'] = def_nom
        pred_convert.loc[i, 'date'] = data.loc[i+n+1, 'trade_date']
    # Data split
    x_train = train.iloc[:train.shape[0]-100]
    x_valid = train.iloc[train.shape[0]-100:]
    y_train = target.iloc[:target.shape[0]-100]
    y_valid = target.iloc[target.shape[0]-100:]
    y_train.drop(['date'], axis=1, inplace=True)
    y_valid.drop(['date'], axis=1, inplace=True)

    # Convert shape of data for LSTM model
    x_train = x_train.to_numpy().reshape((x_train.shape[0], n, 2))
    x_valid = x_valid.to_numpy().reshape((x_valid.shape[0], n, 2))
    return x_train, x_valid, y_train, y_valid, target, pred_convert


In [ ]:
n = 25  # chunk from the dataset used to train the model
x, x_val, y, y_val, target, pred_convert = data_preproc_and_split(btc_data, n)

x = np.asarray(x).astype('float32')
y = np.asarray(y).astype('float32')
x_val = np.asarray(x_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')

In [ ]:
# Define Model

In [ ]:
model = Sequential()
model.add(LSTM(32,
               return_sequences=True, input_shape=(25, 2)))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
cb = TrainCallback() # init Callback
model.fit(x, y,
          batch_size=512,
          validation_data=(x_val, y_val),
          epochs=25, shuffle=False, verbose=2,
          callbacks=[cb]) #Add Callbacks to training